In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds
from HUGS.Util import load_hugs_json

import ipyleaflet as ipl
import ipywidgets as ipw

from pathlib import Path

# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

In [ ]:
center = [51.506815, -0.127710]
zoom = 10

# url = "https://api.mapbox.com/styles/v1/hugs/ck5mpl49v0ng71iokprtni7ra/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiaHVncyIsImEiOiJjazVtbzdxd20wNDFjM21wZDFtZzU1ZG5jIn0.qILQ7iSoXbV78pmZZF8Lcg"
url = "https://api.mapbox.com/styles/v1/hugs/ck5ms14cu3jzo1imv5iizbgz8/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiaHVncyIsImEiOiJjazVtbzdxd20wNDFjM21wZDFtZzU1ZG5jIn0.qILQ7iSoXbV78pmZZF8Lcg"

mapbox_map = ipl.TileLayer(url=url, opacity=1)
map_london = ipl.Map(center=center, zoom=zoom)
map_london.add_layer(mapbox_map)


marker_legend = ipw.HTML(value="<img src='marker-icon-blue.png'>   Current site<br><img src='marker-icon-red.png'>   Future site")
marker_control = ipl.WidgetControl(widget=marker_legend, position="topright")
map_london.add_control(marker_control)

lghg_data = load_hugs_json("lghg_sites.json")

for site in lghg_data["current"]:
    curr_site = lghg_data["current"][site]
    
    lat = curr_site["latitude"]
    long = curr_site["longitude"]
    site_name = curr_site["long_name"]
    
    marker = ipl.Marker(location=(lat, long), draggable=False)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
for site in lghg_data["future"]:
    fut_site = lghg_data["future"][site]
    
    lat = fut_site["latitude"]
    long = fut_site["longitude"]
    site_name = fut_site["long_name"]
    
    # Here we want a green icon
    icon = ipl.Icon(icon_url='marker-icon-red.png', icon_size=[25, 40], icon_anchor=[12,15])
    
    marker = ipl.Marker(location=(lat, long), draggable=False, icon=icon)
    marker.popup = ipw.HTML(value=f"Site: {site_name} ({site})")
    map_london.add_layer(marker)
    
    
map_london

#### Figure 4. Map of London showing sites that are currently part (blue markers) and are soon to be part (red markers) of the London GreeHouse Gas network run by the Atmospheric Chemistry Research Group at the University of Bristol. Overlain on the map is data showing tree density / canopy cover across the boroughs of London.